In [2]:
from qat.interop.qiskit import qlm_to_qiskit
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

from qat.interop.qiskit import BackendToQPU
from qiskit import *

In [3]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [4]:
provider = IBMQ.get_provider(group='open')

In [5]:
nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

# Conversion
qiskit_circuit = qlm_to_qiskit(qlm_circuit)
print(qiskit_circuit)

      ┌───┐     ┌─┐   
q0_0: ┤ H ├──■──┤M├───
      └───┘┌─┴─┐└╥┘┌─┐
q0_1: ─────┤ X ├─╫─┤M├
           └───┘ ║ └╥┘
c0: 2/═══════════╩══╩═
                 0  1 


In [6]:
qlm_job = qlm_circuit.to_job(nbshots=1024)

# Runtime using ibmq_qasm_simulator
backend = provider.backend.ibmq_qasm_simulator
qpu = BackendToQPU(backend)

# Execute via Qiskit Runtime
result = qpu.submit(qlm_job)
for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))

State: |00>	 probability: 0.5283203125
State: |11>	 probability: 0.4716796875
